In [1]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  
model=init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("What is the capital of France?")
print(response)

content='<think>\nOkay, the user is asking, "What is the capital of France?" Hmm, this seems like a straightforward question. Let me start by recalling basic geography. France is a country in Western Europe. I remember that major cities in France include Paris, Lyon, Marseille, and others. Now, capitals are usually the largest or most politically significant city. Paris rings a bell as the capital. Let me verify that.\n\nI think Paris is indeed the capital of France. It\'s where the government is based, and it\'s the most well-known city associated with France. But wait, could there be any confusion? I know that some countries have different cities for their capitals versus their largest cities. For example, in the UK, London is both the capital and the largest city. In France, Paris is both the capital and the largest city. \n\nLet me double-check to make sure I\'m not mixing up any details. I recall that Paris is located in the northern part of France, along the Seine River. It\'s a 

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which is Boston here. I'll call the function with location set to Boston. Make sure the JSON is correctly formatted with the name and arguments.\n", 'tool_calls': [{'id': '5r8w57zfq', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 154, 'total_tokens': 240, 'completion_time': 0.148454486, 'completion_tokens_details': {'reasoning_tokens': 62}, 'prompt_time': 0.006121622, 'prompt_tokens_details': None, 'queue_time': 0.158826778, 'total_time': 0.154576108}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019c5821-539a-72c3-9

Tools Executions

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny.


In [5]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. Let me check the function\'s parameters. It requires a location, which is Boston here. I\'ll call the function with location set to "Boston". Make sure the JSON is correctly formatted with the name and arguments.\n', 'tool_calls': [{'id': 'j3gsgwmyt', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 153, 'total_tokens': 241, 'completion_time': 0.154485054, 'completion_tokens_details': {'reasoning_tokens': 64}, 'prompt_time': 0.006222402, 'prompt_tokens_details': None, 'queue_time': 0.048079248, 'total_time': 0.160707456}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_call